In [ ]:
dtypes = {
    'date': 'object',
    'customer_id': 'object',
    'n_orders': 'int',
    'customer_gender': 'object',
    'customer_age': 'int',
    'customer_created_at': 'object',
    'customer_first_order_paid': 'object',
    'channel_partner_name': 'object',
    'sku_config': 'object',
    'is_campaign': 'bool',
    'product_name': 'object',
    'product_gender': 'object',
    'product_color': 'object',
    'product_brand': 'object',
    'product_medium_image_url': 'object',
    'cmc_category_bp': 'object', 
    'cmc_division_bp': 'object',
    'cmc_business_unit_bp': 'object',
    'google_product_category': 'object',
    'product_original_price': 'float',
    'payment_method_name': 'object',
    'shipping_condition': 'object',
    'product_discount': 'object',
    'shipping_discount': 'object',
    'sale_value': 'float',
    'is_coupon': 'bool',
    'device_name': 'object',
    'platform_name': 'object',
    'delivery_city': 'object',
    'delivery_state_code': 'object',
    'delivery_country_region': 'object',
    'planning_cluster': 'object',
    'planning_age': 'object',
    'ticketrange_planning': 'object',
    'originalpricerange_planning': 'object'
}

In [ ]:
import pandas as pd
from dask import delayed

@delayed
def load(filename):
    return pd.read_parquet(filename, engine='pyarrow')

@delayed
def clean(data):
    return data

@delayed
def analyze(sequence_of_data):
    pass

@delayed
def store(result):
    with open(..., 'w') as f:
        f.write(result)

In [ ]:
files = Path(".").glob(DATA_PATH)

loaded = dd.DataFrame([load(i) for i in files])
cleaned = [clean(i) for i in loaded]
analyzed = analyze(cleaned)
stored = store(analyzed)

In [5]:
DATA_PATH = '../test_dir/input/data/training/part*parquet'

In [6]:
import dask.dataframe as dd
import lens
from pathlib import Path

In [7]:
df = dd.read_parquet(DATA_PATH, columns=["product_medium_image_url"])

In [8]:
help(df.to_parquet, engine='pyarrow', compression='gzip')

Help on method to_parquet in module dask.dataframe.core:

to_parquet(path, *args, **kwargs) method of dask.dataframe.core.DataFrame instance
    Store Dask.dataframe to Parquet files
    
    Notes
    -----
    Each partition will be written to a separate file.
    
    Parameters
    ----------
    df : dask.dataframe.DataFrame
    path : string
        Destination directory for data.  Prepend with protocol like ``s3://``
        or ``hdfs://`` for remote data.
    engine : {'auto', 'fastparquet', 'pyarrow'}, default 'auto'
        Parquet library to use. If only one library is installed, it will use
        that one; if both, it will use 'fastparquet'.
    compression : string or dict, optional
        Either a string like ``"snappy"`` or a dictionary mapping column names
        to compressors like ``{"name": "gzip", "values": "snappy"}``. The
        default is ``"default"``, which uses the default compression for
        whichever engine is selected.
    write_index : boolean, op

In [ ]:
df.to_parquet('test', partition_on=False)

In [ ]:
ids = unique_url.compute()

In [36]:
ls = lens.summarise(df)

TypeError: Can only summarise a Pandas DataFrame

In [ ]:
customer_id = df.customer_id.compute()

In [13]:
df_temp.nunique()

dd.Scalar<series-..., dtype=int64>

In [ ]:
df_temp.compute()

In [ ]:
df.describe().compute()

In [ ]:
df.head()

In [ ]:
df.customer_id.nunique().compute()

In [16]:
%matplotlib inline
df.n_orders.value_counts().compute()

44     113751
45     106992
46      96678
47      95372
48      87765
49      87665
50      85496
51      81521
52      76342
54      72305
53      72055
55      71065
56      63404
58      60705
57      60027
61      58396
59      56529
60      55745
62      54535
64      50506
63      49273
65      44717
66      44713
69      44093
68      42123
70      41963
67      39645
74      38039
79      36553
80      35300
        ...  
310       528
257       519
206       507
243       458
428       449
554       436
372       433
214       430
270       428
377       417
313       416
197       394
233       392
306       374
319       365
247       345
286       329
225       319
261       276
367       265
318       250
349       211
395       190
314       181
458       171
237       145
347        86
293        83
405        81
300        37
Name: n_orders, Length: 313, dtype: int64

In [18]:
df.shape

(3534847, 35)

In [17]:
# core modules
import logging
import sys

import pandas as pd


logging.basicConfig(format='%(asctime)s %(levelname)s %(message)s',
                    level=logging.DEBUG,
                    stream=sys.stdout)


def describe_pandas_df(df, dtype=None):
    """
    Show basic information about a pandas dataframe.

    Parameters
    ----------
    df : Pandas Dataframe object
    dtype : dict
        Maps column names to types
    """
    if dtype is None:
        dtype = {}
    print("Number of datapoints: {datapoints}".format(datapoints=len(df)))
    column_info = {'int': [], 'float': [], 'category': [], 'other': []}
    float_types = ['float64']
    integer_types = ['int64', 'uint8']
    other_types = ['object', 'category']
    column_info_meta = {}
    for column_name in df:
        column_info_meta[column_name] = {}
        counter_obj = df[column_name].groupby(df[column_name]).count()
        value_list = list(counter_obj.keys())
        value_count = len(value_list)
        is_suspicious_cat = (value_count <= 50 and
                             str(df[column_name].dtype) != 'category' and
                             column_name not in dtype)
        if is_suspicious_cat:
            logging.warning("Column '{}' has only {} different values ({}). "
                            "You might want to make it a 'category'"
                            .format(column_name,
                                    value_count,
                                    value_list))
        top_count_val = counter_obj[value_list[0]]
        column_info_meta[column_name]['top_count_val'] = top_count_val
        column_info_meta[column_name]['value_list'] = value_list
        column_info_meta[column_name]['value_count'] = value_count

        if df[column_name].dtype in integer_types:
            column_info['int'].append(column_name)
        elif df[column_name].dtype in float_types:
            column_info['float'].append(column_name)
        elif str(df[column_name].dtype) == 'category':
            column_info['category'].append(column_name)
        elif str(df[column_name].dtype) in other_types:
            column_info['other'].append(column_name)
        else:
            print("!!! describe_pandas_df does not know type '{}'"
                  .format(df[column_name].dtype))

    column_name_len = max(len(column_name) for column_name in df)

    print("\n## Integer Columns")
    print("{column_name:<{column_name_len}}: Non-nan  mean   std   min   25%  "
          " 50%   75%   max"
          .format(column_name_len=column_name_len,
                  column_name="Column name"))
    for column_name in column_info['int']:
        print("{column_name:<{column_name_len}}: {non_nan:>7}  "
              "{mean:0.2f}  {std:>4.2f}  "
              "{min:>4.0f}  {q25:>4.0f}  {q50:>4.0f}  {q75:>4.0f}  {max:>4.0f}"
              .format(column_name_len=column_name_len,
                      column_name=column_name,
                      non_nan=sum(df[column_name].notnull()),
                      mean=df[column_name].mean(),
                      std=df[column_name].std(),
                      min=df[column_name].min(),
                      q25=df[column_name].quantile(0.25),
                      q50=df[column_name].quantile(0.50),
                      q75=df[column_name].quantile(0.75),
                      max=max(df[column_name])))

    print("\n## Float Columns")
    print("{column_name:<{column_name_len}}: Non-nan   mean    std    min    "
          "25%    50%    75%    max"
          .format(column_name_len=column_name_len,
                  column_name="Column name"))
    for column_name in column_info['float']:
        print("{column_name:<{column_name_len}}: {non_nan:>7}  "
              "{mean:5.2f}  {std:>4.2f}  "
              "{min:>5.2f}  {q25:>5.2f}  {q50:>5.2f}  {q75:>5.2f}  {max:>5.2f}"
              .format(column_name_len=column_name_len,
                      column_name=column_name,
                      non_nan=sum(df[column_name].notnull()),
                      mean=df[column_name].mean(),
                      std=df[column_name].std(),
                      min=df[column_name].min(),
                      q25=df[column_name].quantile(0.25),
                      q50=df[column_name].quantile(0.50),
                      q75=df[column_name].quantile(0.75),
                      max=max(df[column_name])))
    print("\n## Category Columns")
    print("{column_name:<{column_name_len}}: Non-nan   unique   top (count)  "
          "rest"
          .format(column_name_len=column_name_len,
                  column_name="Column name"))
    for column_name in column_info['category']:
        # print(df[column_name].describe())
        rest_str = str(column_info_meta[column_name]['value_list'][1:])[:40]
        print("{column_name:<{column_name_len}}: {non_nan:>7}   {unique:>6}   "
              "{top} ({count})  {rest}"
              .format(column_name_len=column_name_len,
                      column_name=column_name,
                      non_nan=sum(df[column_name].notnull()),
                      unique=len(df[column_name].unique()),
                      top=column_info_meta[column_name]['value_list'][0],
                      count=column_info_meta[column_name]['top_count_val'],
                      rest=rest_str))

    print("\n## Other Columns")
    print("{column_name:<{column_name_len}}: Non-nan   unique   top (count)"
          .format(column_name_len=column_name_len,
                  column_name="Column name"))
    for column_name in column_info['other']:
        rest_str = str(column_info_meta[column_name]['value_list'][1:])[:40]
        print("{column_name:<{column_name_len}}: {non_nan:>7}   {unique:>6}   "
              "{top} ({count})"
              .format(column_name_len=column_name_len,
                      column_name=column_name,
                      non_nan=sum(df[column_name].notnull()),
                      unique=len(df[column_name].unique()),
                      top=column_info_meta[column_name]['value_list'][0],
                      count=column_info_meta[column_name]['top_count_val']))


def get_parser():
    """Get parser object for exploratory data analysis."""
    from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter
    parser = ArgumentParser(description=__doc__,
                            formatter_class=ArgumentDefaultsHelpFormatter)
    parser.add_argument("-f", "--file",
                        dest="filename",
                        help="read this csv file",
                        metavar="FILE",
                        required=True)
    return parser


dtypes = {
    'date': 'str',
    'customer_created_at': 'str',
    'customer_first_order_paid': 'str',
    'is_coupon': 'str',
    'is_campaign': 'str',
    
    'customer_id': 'category',
    'customer_gender': 'category',
    'channel_partner_name': 'category',
    'sku_config': 'category',
    'product_name': 'category',
    'product_gender': 'category',
    'product_color': 'category',
    'product_brand': 'category',
    'product_medium_image_url': 'category',
    'cmc_category_bp': 'category', 
    'cmc_division_bp': 'category',
    'cmc_business_unit_bp': 'category',
    'google_product_category': 'category',
    'payment_method_name': 'category',
    'shipping_condition': 'category',
    'device_name': 'category',
    'platform_name': 'category',
    'delivery_city': 'category',
    'delivery_state_code': 'category',
    'delivery_country_region': 'category',
    'planning_cluster': 'category',
    'planning_age': 'category',
    'ticketrange_planning': 'category',
    'originalpricerange_planning': 'category',
    
    'customer_age': 'int64',
    'n_orders': 'int64',
    
    'sale_value': 'float64', 
    'product_original_price': 'float64',
    'product_discount': 'float64',
    'shipping_discount': 'float64',
}
parse_dates = ['date', 'customer_created_at', 'customer_first_order_paid']

df = pd.read_csv(DATA_PATH, dtype=dtypes, parse_dates=parse_dates)

describe_pandas_df(df.drop([
    "product_medium_image_url", "product_name", "n_orders", "customer_age",
    'date', 'customer_created_at', 'customer_first_order_paid'],
    axis=1), dtype=dtypes)

Number of datapoints: 3534847

## Integer Columns
Column name                : Non-nan  mean   std   min   25%   50%   75%   max

## Float Columns
Column name                : Non-nan   mean    std    min    25%    50%    75%    max
product_original_price     : 3534847  656.51  219299.42   4.00  79.00  130.00  229.00  99999999.00
product_discount           : 3534847  10.45  25.08   0.00   0.00   0.00   9.10  1750.49
shipping_discount          : 3534847  1278.50  342256.24  -55.98   1.41   3.19   6.19  99999999.99
sale_value                 : 3534847  100.17  83.14   0.00  44.99  76.99  130.19  3998.00

## Category Columns
Column name                : Non-nan   unique   top (count)  rest
customer_id                : 3534847    23664   1445343 (742)  ['1601186', '1810075', '1997117', '21173
customer_gender            : 3531028        3   female (2996744)  ['male']
channel_partner_name       : 3534847       74   APPLIFT (46938)  ['AUTOMATED', 'BING', 'CRITEO', 'DIRECT'
sku_config         

In [55]:
describe_pandas_df(df.drop_duplicates(["customer_id"], keep='first')[["n_orders", "customer_age"]], dtype=dtypes)

Number of datapoints: 23664

## Integer Columns
Column name : Non-nan  mean   std   min   25%   50%   75%   max
n_orders    :   23664  70.83  38.37    43    49    58    78   697
customer_age:   23664  39.50  11.16     0    32    38    46   119

## Float Columns
Column name : Non-nan   mean    std    min    25%    50%    75%    max

## Category Columns
Column name : Non-nan   unique   top (count)  rest

## Other Columns
Column name : Non-nan   unique   top (count)
